In [6]:
from copy import deepcopy
import gurobipy as gp
from gurobipy import GRB
import torch
import torch.nn as nn
import numpy as np
import os
import math
import time
from cutting_plane_framework import CuttingPlaneMethod


In [7]:
class NaiveCuttingPlaneTreeAlgorithm(CuttingPlaneMethod):
    def __init__(self, instanceName, maxIteration=100, OutputFlag=0, Threads=1, MIPGap=0.0, TimeLimit=3600, MIPFocus=2, cglp_OutputFlag=0, cglp_Threads=1, cglp_MIPGap=0.0, cglp_TimeLimit=100, cglp_MIPFocus=0, addCutToMIP=False, number_branch_var=2, normalization='SNC', additional_param=None):
        super().__init__(instanceName, maxIteration, OutputFlag, Threads, MIPGap, TimeLimit, MIPFocus, cglp_OutputFlag, cglp_Threads, cglp_MIPGap, cglp_TimeLimit, cglp_MIPFocus, addCutToMIP, number_branch_var, normalization)
        self.additional_param = additional_param
    
    def variable_selection(self):
        #TODO::add ML model to choose the variable to branch
        # choose the variable to branch: Maximum Fractionality Rule
        number_of_candidates = self.number_branch_var                                                                       # the number of variables that are chosen to branch, so the number of nodes in the branching tree is 2^number_of_candidates
        number_of_noninteger = len(self.non_integer_vars[self.iteration - 1])
        number_of_nonbinary = len(self.non_binary_vars[self.iteration - 1])
        self.branchVar[self.iteration-1] = {}
        if number_of_noninteger > 0:
            list1 = sorted(self.non_integer_vars[self.iteration-1].items(), key=lambda x: x[1], reverse=True)[:number_of_candidates]   # find the integer variables that have the largest distance to the nearest integer
            if len(list1) <= number_of_candidates:
                for item in list1:
                    self.branchVar[self.iteration-1][item[0]] = item[1] # 
            else:
                for item in list1[0:number_of_candidates]:
                    self.branchVar[self.iteration-1][item[0]] = item[1]

        if number_of_nonbinary > 0: 
            list2 = sorted(self.non_binary_vars[self.iteration-1].items(), key=lambda x: x[1], reverse=True)[:number_of_candidates]    # find the binary variables that have the largest distance to {0,1}
            if len(list2) <= number_of_candidates:
                for item in list2:
                    self.branchVar[self.iteration-1][item[0]] = item[1]
            else:
                for item in list2[0:number_of_candidates]:
                    self.branchVar[self.iteration-1][item[0]] = item[1]


    def branching_tree_building(self, node, level, varInfo):
        if level == len(self.branchVar[self.iteration-1]):
            return
        else:
            varName, info = list(varInfo.items())[level]
            pos = self.varName_map_position[varName]

            left_node = {}
            left_node['LB'] = deepcopy(self.nodeSet[node]['LB'])
            left_node['LB'][pos] = info['upper']
            left_node['UB'] = deepcopy(self.nodeSet[node]['UB'])
            left_node['trace'] = deepcopy(self.nodeSet[node]['trace'])
            left_node['trace'].append('l')
            
            right_node = {}
            right_node['LB'] = deepcopy(self.nodeSet[node]['LB'])
            right_node['UB'] = deepcopy(self.nodeSet[node]['UB'])
            right_node['UB'][pos] = info['lower']
            right_node['trace'] = deepcopy(self.nodeSet[node]['trace'])
            right_node['trace'].append('r')

            left_node_ind = max(self.nodeSet.keys()) + 1
            right_node_ind = left_node_ind + 1
            self.nodeSet[left_node_ind] = left_node
            self.nodeSet[right_node_ind] = right_node
            del self.nodeSet[node]  

            self.branching_tree_building(left_node_ind, level+1, varInfo)
            self.branching_tree_building(right_node_ind, level+1, varInfo)
     
    def branching_tree(self):
        varInfo = {}
        for varName in self.branchVar[self.iteration-1].keys():
            varInfo[varName] = {}
            varInfo[varName]['val'] = self.lp_relaxation.getVarByName(varName).x
            varInfo[varName]['lower'] = math.floor(varInfo[varName]['val'])
            varInfo[varName]['upper'] = math.ceil(varInfo[varName]['val'])
        
        self.nodeSet = {}
        self.nodeSet[0] = {}
        self.nodeSet[0]['LB'] = deepcopy(self.LB)
        self.nodeSet[0]['UB'] = deepcopy(self.UB)
        self.nodeSet[0]['trace'] = []
        self.branching_tree_building(0, 0, varInfo) 
    
    def solve(self):
        time_init = time.time()
        while self.iteration <= self.maxIteration:
            iter_begin = time.time()
            self.master_problem()
            if self.OPT == True:
                self.print_iteration_info()
                return
            self.variable_selection()
            self.branching_tree()
            ready_to_cut = time.time()
            self.cut_generation()
            iter_end = time.time()
            overall = iter_end - time_init
            iteration_time = iter_end - iter_begin
            cut_time = iter_end - ready_to_cut
            self.print_iteration_info(cut_time, iteration_time, overall)
        

    


In [12]:
instanceName = '50v-10'
cpt = NaiveCuttingPlaneTreeAlgorithm(instanceName, maxIteration=100, OutputFlag = 0, Threads = 1, MIPGap = 0.0, TimeLimit = 300, number_branch_var = 1, normalization = 'SNC')
cpt.solve()

Read MPS format model from file benchmark/50v-10.mps.gz
Reading time = 0.02 seconds
50v-10: 233 rows, 2013 columns, 2745 nonzeros
This problem has 1647 integer variables and 0 binary variables.
The optimal value of LP relaxation is 2879.0656868536717.
---------------------------------------------------------------------------------------------------------------------------------
|  Iter  |  # fractional var  |  current value  |  Relative Improvement  |  Overall Improvement  |  Iter Time  |  Overall Time  |
---------------------------------------------------------------------------------------------------------------------------------
|       1|                  29|        2882.1282|                 0.1063 |                0.1064 |      0.2966 |         0.6118 |
|       2|                  29|        2885.2011|                 0.1065 |                0.2131 |      0.2909 |         0.9027 |
|       3|                  29|        2888.0303|                 0.0980 |                0.3114 |